In [4]:
import pandas as pd

dt = pd.read_csv('./data/clsoet_share_bags.csv').drop('Unnamed: 0',axis=1)
cs_dt = pd.read_excel('./data/클로젯셰어_과제DB 대체 자료.xlsx')

dt.columns = ['brandnm', 'goodsnm', '4days', '7days']

display(dt)
display(cs_dt)

FileNotFoundError: [Errno 2] File ./data/clsoet_share_bags.csv does not exist: './data/clsoet_share_bags.csv'

# dt와 cs_dt join : left

In [ ]:
merge_dt = pd.merge(dt, cs_dt, how='right', on=['brandnm','goodsnm'])
merge_dt

In [ ]:
merge_dt.isnull().sum()

In [ ]:
#merge_dt[merge_dt['brandnm'].isnull()]

# inner join

In [ ]:
inner_dt = pd.merge(cs_dt, dt, how='inner', on='goodsnm')
inner_dt.drop('brandnm_y', axis=1, inplace=True)

In [ ]:
an = inner_dt.app_yn.fillna('N')
inner_dt['app_yn'] = an
inner_dt

In [ ]:
inner_dt.columns = ['brandnm', 'goodsnm', 'USER', 'reg_date' ,'order_date', 'p_type', 'sub_type', 'app_yn', '4days', '7days']
inner_dt

In [ ]:
inner_dt.to_csv('./data/bags_inner_dt.csv')

## 브랜드별 일회권 평균 값 구하기

In [ ]:
inner_dt.brandnm.value_counts()

In [ ]:
inner_dt.query('brandnm == "Chanel"')

In [ ]:
def get_font_family():
    """
    시스템 환경에 따른 기본 폰트명을 반환하는 함수
    """
    import platform
    system_name = platform.system()
    # colab 사용자는 system_name이 'Linux'로 확인

    if system_name == "Darwin" :
        font_family = "AppleGothic"
    elif system_name == "Windows":
        font_family = "Malgun Gothic"
    else:
        # Linux
        # colab에서는 runtime을 <꼭> 재시작 해야함.
        # 런타임을 재시작 하지 않고 폰트 설치를 하면 기본 설정 폰트가 로드되어 한글이 깨짐.
        !apt-get update -qq
        !apt-get install fonts-nanum -qq  > /dev/null

        import matplotlib.font_manager as fm

        fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
        font = fm.FontProperties(fname=fontpath, size=9)
        fm._rebuild()
        font_family = "NanumBarunGothic"
    return font_family

In [ ]:
# 시각화를 위한 폰트설정
# 위에서 만든 함수를 통해 시스템 폰트를 불러와서 font_family 라는 변수에 할당.
a = get_font_family()
# 폰트설정
import matplotlib.pyplot as plt 
plt.rc("font", family = a)
# 마이너스폰트 설정
plt.rc("axes", unicode_minus=False)
# ggplot으로 그래프 스타일 설정 / 개인 자유
plt.style.use("ggplot")

import seaborn as sns
%matplotlib inline

import warnings
warnings.filterwarnings(action='ignore')

#한글 폰트 설정
plt.rcParams['font.family'] = 'HYGothic-Extra'

In [ ]:
plt.figure(figsize=(15,5))
plt.subplot(121)
plt.scatter(x='brandnm', y='4days', data=inner_dt)
plt.xticks(rotation=-90)

plt.subplot(122)
plt.scatter(x='brandnm', y='7days', data=inner_dt)
plt.xticks(rotation=-90)
plt.show()

- 일반적으로 7일권이 4일권보다 1만원 정도 비쌈

In [ ]:
brands_mean = inner_dt.groupby('brandnm').mean()
brands_mean.reset_index(inplace=True)
brands_mean

# 데이터 합치기

In [ ]:
#concat
concat_dt = pd.concat([cs_dt,inner_dt], axis=0)
concat_dt

In [3]:
concat_dt.query('sub_type == "BA"').isnull().sum()

NameError: name 'concat_dt' is not defined

In [ ]:
# 중복 제거 keep = last

concat_dt = concat_dt.drop_duplicates(subset=['brandnm','goodsnm','USER','reg_date',
                                  'order_date', 'p_type', 'sub_type', 'app_yn'], keep='last')

In [ ]:
concat_dt

In [ ]:
concat_dt.query('sub_type == "BA"').isnull().sum()

In [ ]:
a = concat_dt.query('sub_type == "BA"')
print(a.brandnm.nunique()) # 47개 브랜드
print(a.brandnm.unique())

In [ ]:
concat_dt.duplicated().sum()

In [ ]:
an = concat_dt.app_yn.fillna('N')
concat_dt['app_yn'] = an
concat_dt

In [ ]:
concat_dt.reset_index(inplace=True)
concat_dt.drop('index', axis=1, inplace=True)

In [ ]:
zero4 = concat_dt['4days'].fillna(0) # 결측치를 0으로 채움
concat_dt['4days'] = zero4

zero7 = concat_dt['7days'].fillna(0) # 결측치를 0으로 채움
concat_dt['7days'] = zero7

In [ ]:
concat_dt

In [ ]:
concat_dt.isnull().sum()

In [ ]:
#concat_dt[concat_dt['brandnm'].isnull()] # TP라서 상관 x

In [ ]:
concat_dt.isnull().sum()

In [ ]:
concat_dt[concat_dt['brandnm'].isnull()] # sub_type이 TP

### 나머지 33개 브랜드 값은??

In [ ]:
excepted = concat_dt.loc[(concat_dt['4days'] == 0) & (concat_dt['sub_type'] == "BA")]
excepted

In [ ]:
excepted.brandnm.unique()

In [ ]:
excepted.brandnm.nunique()

In [ ]:
dt_brands = dt.groupby('brandnm').mean() #크롤링한 데이터를 바탕으로 한 평균 값
dt_brands.reset_index(inplace=True)
#dt_brands.drop('index', axis=1, inplace=True)
dt_brands

In [ ]:
# brands_mean = inner_dt.groupby('brandnm').mean()
# brands_mean.reset_index(inplace=True)

brands_mean

In [ ]:
brands_means = pd.concat([dt_brands, brands_mean], axis=1)
# dt_brands는 크롤링한 데이터의 평균값
# brands_mean은 inner join한 데이터의 평균값
brands_means

- brands_mean에 없는 데이터를 dt_brands에서 가져옴
    - FONTANA,SALVATORE FERRAGAMO, TRAVEL MATE <- 근데 얘네는 concat_data에 없음

In [ ]:
## 평균 값을 4/7days nan 값에 넣기

for i in range(len(concat_dt)):
    if (concat_dt['sub_type'][i] == 'BA' and concat_dt['brandnm'][i] == 'Balenciaga'):
        concat_dt['4days'][i] = 49000
        concat_dt['7days'][i] = 59000
        
    elif (concat_dt['sub_type'][i] == 'BA' and concat_dt['brandnm'][i] == 'Celine'):
        concat_dt['4days'][i] = 41500
        concat_dt['7days'][i] = 51500

    elif (concat_dt['sub_type'][i] == 'BA' and concat_dt['brandnm'][i] == 'Chanel'):
        concat_dt['4days'][i] = 45800
        concat_dt['7days'][i] = 55800

    elif (concat_dt['sub_type'][i] == 'BA' and concat_dt['brandnm'][i] == 'Delvaux'):
        concat_dt['4days'][i] = 59000
        concat_dt['7days'][i] = 69000
    
    elif (concat_dt['sub_type'][i] == 'BA' and concat_dt['brandnm'][i] == 'Fendi'):
        concat_dt['4days'][i] = 46500
        concat_dt['7days'][i] = 56500
    
    elif (concat_dt['sub_type'][i] == 'BA' and concat_dt['brandnm'][i] == 'Gucci'):
        concat_dt['4days'][i] = 46450
        concat_dt['7days'][i] = 56450
        
    elif (concat_dt['sub_type'][i] == 'BA' and concat_dt['brandnm'][i] == 'Hermes'):
        concat_dt['4days'][i] = 39000
        concat_dt['7days'][i] = 49000
    
    elif (concat_dt['sub_type'][i] == 'BA' and concat_dt['brandnm'][i] == 'Louis Vuitton'):
        concat_dt['4days'][i] = 45950
        concat_dt['7days'][i] = 55950
        
    elif (concat_dt['sub_type'][i] == 'BA' and concat_dt['brandnm'][i] == 'Miu Miu'):
        concat_dt['4days'][i] = 39000
        concat_dt['7days'][i] = 49000
    
    elif (concat_dt['sub_type'][i] == 'BA' and concat_dt['brandnm'][i] == 'Prada'):
        concat_dt['4days'][i] = 49000
        concat_dt['7days'][i] = 59000
    
    elif (concat_dt['sub_type'][i] == 'BA' and concat_dt['brandnm'][i] == 'Rimowa'):
        concat_dt['4days'][i] = 39000
        concat_dt['7days'][i] = 49000
    
    elif (concat_dt['sub_type'][i] == 'BA' and concat_dt['brandnm'][i] == 'Saint Laurent'):
        concat_dt['4days'][i] = 49000
        concat_dt['7days'][i] = 59000
    
    elif (concat_dt['sub_type'][i] == 'BA' and concat_dt['brandnm'][i] == 'Valentino'):
        concat_dt['4days'][i] = 39000
        concat_dt['7days'][i] = 49000
    
    elif (concat_dt['sub_type'][i] == 'BA' and concat_dt['brandnm'][i] == 'Yves Saint Laurent'):
        concat_dt['4days'][i] = 49000
        concat_dt['7days'][i] = 59000

    
concat_dt.query('sub_type == "BA"')

In [ ]:
excepted.brandnm.unique()

In [ ]:
excepted.brandnm.nunique()

## 33개 브랜드 크롤링?

In [ ]:
### 검색 리스트 생성

import numpy as np

search_list = excepted['brandnm'].unique()
s_df = pd.DataFrame(search_list)
s_df.columns = ['keyword']

search = list(np.array(s_df['keyword'].tolist()))

search

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys # 엔터키 입력용

import pandas as pd
import time

In [ ]:
# 드라이버
driver = webdriver.Chrome('C:/Temp/chromedriver') # 웹드라이버 객체 생성

# 구글을 열어서 키워드 입력 후 이동
driver.get('https://www.closetshare.com/theclozet/main/index') # 구글 홈페이지 내용 렌더링
time.sleep(1)

# 리스트 선언
clothes_price_list_4 = []
clothes_price_list_7 = []
brandnm_list = []
goodsnm_list = []


# popup 제거
popup = driver.find_element_by_css_selector("[id = 'notshow']")
popup.click()
time.sleep(1)

# 로그인 페이지 이동
login = driver.find_element_by_css_selector("[id = 'account_login']")
login.click()
time.sleep(1)

# 로그인_이메일
login_2 = driver.find_element_by_css_selector('#loginForm > ul > li.ic-email.ic-login-email > a > span')
login_2.click()

# 아이디
email = driver.find_element_by_css_selector("[id = 'email']")
email.send_keys('rjs4770@naver.com') # 개인 아이디 입력

# 패스워드
password = driver.find_element_by_css_selector("[id = 'passwd']")
password.send_keys('Tjdwns4845!') # 개인 패스워드 입력

# 로그인 엔터
password.send_keys(Keys.ENTER)

# 렌더링 1초 기다리자
time.sleep(2)

# 가방 카테고리 클릭
bags = driver.find_element_by_css_selector('#cshrHeaderWeb > nav > ul > li:nth-child(2) > a')
bags.click()
time.sleep(2)

# 필터링
filtering = driver.find_element_by_css_selector('#days_flag')
filtering.click()
time.sleep(1)

# 대여가능만 클릭
driver.find_element_by_css_selector('[data ="clist_filter_s1"]').click()


count = 0
# 크롤링을 진행해봅시다

for i in range(5):
    # 검색창에 값 입력f
    driver.find_element_by_xpath('//*[@id="frmList"]/section/section[1]/div[1]/article[1]/input').send_keys(search[i]) # 검색어 입력
    driver.find_element_by_xpath('//*[@id="goodsnmSesrch"]').click() # 검색 버튼 클릭
    time.sleep(1)
    
    for clothes_num in range(1,37) :
        try : 
            # 브랜드명 수집 코드 입니다.
            brandnm = driver.find_element_by_css_selector("#prod_list_item > a:nth-child("+str(clothes_num)+") > span > span.brand")
            brandnm_list.append(brandnm.text)
            print("브랜드 :",brandnm.text)

            # 아이템명 수집 코드입니다.
            goodsnm = driver.find_element_by_css_selector('#prod_list_item > a:nth-child('+str(clothes_num)+') > span > span.cont')
            goodsnm_list.append(goodsnm.text)
            print("아이템 :",goodsnm.text)


            # 한 페이지 내 36개의 아이템을 indexing하여 클릭하기 위한 코드입니다.
            clothes_index = driver.find_element_by_css_selector('#prod_list_item > a:nth-child('+str(clothes_num)+') > span > span.status.noto_m')
            clothes_index.click()
            time.sleep(2)

            # 4일권 / 7일권 수집
            # 가끔가다 보면 대여가 불가능한 제품들은 4일권과 7일권에 대한 가격 정보가 없습니다.
            # 때문에 try, except 구문으로 가격이 없는 것은 0으로 대체합니다.
            try : 
                price = driver.find_element_by_css_selector('#goodsValue_daily')
                clothes_price_list_4.append(price.text.split('\n')[0][5:11])
                clothes_price_list_7.append(price.text.split('\n')[1][5:11])
                print("4일권 :",price.text.split('\n')[0][5:11])
                print("7일권 :",price.text.split('\n')[1][5:11])
            except :
                clothes_price_list_4.append(0)
                clothes_price_list_7.append(0)
                print("4일권 :",0)
                print("7일권 :",0)

            # 뒤로 돌아가서 다른 아이템 정보를 가져올 준비를 합니다.
            go_home = driver.find_element_by_xpath('//*[@id="cshrHeader"]/h1/a')
            go_home.click()
            time.sleep(1)
            
        except :
            pass

        # 전체 아이템 중 몇 번째인지 확인하는 코드입니다.
        count += 1
        print("=====================================", count, "번째")

#     try :
#         # 만일 두 아이템의 이름이 정확하게 일치하지 않으면
#         if items[i] != driver.find_element_by_xpath('//*[@id="prod_list_item"]/a/span/span[3]').text :
#             price_4days.append('not same item')
#             price_7days.append('not same item')
            
#             item_nm = items[i]
#             bag_name.append(item_nm)
            
#             print(bag_name[i])
#             print("4일권 :",price_4days[i])
#             print("7일권 :",price_7days[i])
            
#             # 홈으로 돌아가서
#             go_home = driver.find_element_by_xpath('//*[@id="cshrHeader"]/h1/a')
#             go_home.click()
            
#             # 다시 가방 카테고리 누르기
#             bags = driver.find_element_by_css_selector('#cshrHeaderWeb > nav > ul > li:nth-child(2) > a')
#             bags.click()
#             time.sleep(2)
            

#         else : # 두 아이템의 이름이 같으면
#             # 이거는 검색 결과가 하나만 나올 때 유효함 ㅜㅜ ↓
#             item_nm = driver.find_element_by_xpath('//*[@id="prod_list_item"]/a/span/span[3]').text
#             bag_name.append(item_nm)
        
#             # 이미지 클릭해서 세부 페이지로 들어가기
#             driver.find_element_by_xpath('//*[@id="prod_list_item"]/a[1]/span').click()
#             time.sleep(1)
        
#             # 4일권/7일권 가격 데이터 가져오기
#             try : 
#                 price = driver.find_element_by_css_selector('#goodsValue_daily')
#                 price_4days.append(price.text.split('\n')[0][5:11])
#                 price_7days.append(price.text.split('\n')[1][5:11])
#                 bag_name.append(item_nm)
                
#                 print(bag_name[i])
#                 print("4일권 :",price.text.split('\n')[0][5:11])
#                 print("7일권 :",price.text.split('\n')[1][5:11])
#                 print("--------------------")
                
#             except : # 가격 데이터 없으면
#                 price_4days.append('no')
#                 price_7days.append('no')
#                 bag_name.append(item_nm)
                
#                 print(bag_name[i])
#                 print("4일권 :", price_4days[i])
#                 print("7일권 :", price_7days[i])
#                 print("--------------------")

#             # 뒤로 돌아가서 다른 아이템 정보를 가져올 준비를 합니다.
#             go_home = driver.find_element_by_xpath('//*[@id="cshrHeader"]/h1/a')
#             go_home.click()
            
#             bags = driver.find_element_by_css_selector('#cshrHeaderWeb > nav > ul > li:nth-child(2) > a')
#             bags.click()
        
#     except :
#         #에러나면(검색 결과가 없으면) pass 홈으로 돌아가서
#         price_4days.append('no result')
#         price_7days.append('no result')
#         bag_name.append(item_nm)
        
#         print(bag_name[i])
#         print("4일권 :", price_4days[i])
#         print("7일권 :", price_7days[i])
#         print("--------------------")
        
#         go_home = driver.find_element_by_xpath('//*[@id="cshrHeader"]/h1/a')
#         go_home.click()

#         # 다시 가방 카테고리 누르기
#         bags = driver.find_element_by_css_selector('#cshrHeaderWeb > nav > ul > li:nth-child(2) > a')
#         bags.click()
#         time.sleep(2)

## cs_dt에서 가방만

In [ ]:
bags = cs_dt.query('sub_type == "BA"').reset_index()
bags.drop('index', axis=1, inplace=True)
bags# 692 rows

### bags와 dt merge

In [ ]:
bag_merge = pd.merge(bags, dt, how='left', on='goodsnm')
bag_merge

In [ ]:
an = bag_merge.app_yn.fillna('N')
bag_merge['app_yn'] = an
bag_merge

In [ ]:
### 4days와 7days가 NaN인 곳 다른 값으로 채운 후 추출
an = bag_merge['4days'].fillna(0)
an2 = bag_merge['7days'].fillna(0)
bag_merge['4days'] = an
bag_merge['7days'] = an2
bag_merge.drop('brandnm_y', axis=1, inplace=True)
bag_merge

In [ ]:
bag_merge.isnull().sum()

In [ ]:
bag_merge.info()

In [ ]:
# 가격 없는 데이터만 추출

no_price = bag_merge.loc[(bag_merge['4days'] == 0) | (bag_merge['7days'] == 0)]
no_price

In [ ]:
import numpy as np

search_list = no_price['brandnm_x'] + ' ' + no_price['goodsnm'] # Series
s_df = pd.DataFrame(search_list)
s_df.columns = ['keyword']

search = list(np.array(s_df['keyword'].tolist()))

In [ ]:
search # 검색할 리스트

In [ ]:
# 아이템 이름만 따로
item_list = no_price['goodsnm']
i_df = pd.DataFrame(item_list)
i_df.columns = ['items']

items = list(np.array(i_df['items'].tolist()))

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys # 엔터키 입력용

import pandas as pd
import time

In [ ]:
print(search[:5], '\n\n')
print(items[:5])

In [ ]:
print(price_4days)
print(price_7days)
print(bag_name)

In [ ]:
# 리스트들 df로 변환 - 209번까지 수집

bag_test = pd.DataFrame([ x for x in zip(bag_name,price_4days,price_7days)])
bag_test.columns = ['Bag','4days','7days']
bag_test
#bag_test.drop_duplicates()